In [ ]:
!pip install pytorch-tabnet

In [ ]:
!pip install lightgbm
!pip install xgboost
#!pip install pytorch-tabnet

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/2022dataset/subset_100k.csv'

In [ ]:
data_update = pd.read_csv(file_path)
data_update

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Texas,Female,Good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,No,...,1.68,113.40,40.35,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
1,Wisconsin,Female,Good,2.0,5.0,Within past year (anytime less than 12 months ...,Yes,8.0,1 to 5,No,...,1.73,104.33,34.97,Yes,Yes,No,Yes,"No, did not receive any tetanus shot in the pa...",No,No
2,Idaho,Male,Poor,7.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,1 to 5,No,...,1.73,104.33,34.97,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
3,Iowa,Female,Good,20.0,10.0,Within past year (anytime less than 12 months ...,No,8.0,All,Yes,...,1.68,127.01,45.19,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
4,South Dakota,Female,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.60,49.90,19.49,Yes,No,Yes,No,"Yes, received Tdap",No,Tested positive using home test without a heal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Utah,Male,Very good,3.0,0.0,5 or more years ago,Yes,7.0,None of them,No,...,1.70,77.11,26.63,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,No
99996,New Mexico,Male,Very good,30.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,All,Yes,...,1.88,90.72,25.68,No,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
99997,Michigan,Female,Very good,2.0,12.0,5 or more years ago,No,7.0,None of them,No,...,1.65,56.70,20.80,Yes,Yes,Yes,Yes,"Yes, received Tdap",Yes,No
99998,Arizona,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.91,106.59,29.37,No,No,Yes,No,"Yes, received tetanus shot, but not Tdap",No,No


In [ ]:
data_update.duplicated().sum()

0

In [ ]:
duplicate_obs = data_update[data_update.duplicated(keep=False)]
duplicate_obs

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos


In [ ]:
data_copy = data_update.copy()
data_copy = data_update.drop_duplicates()
data_copy

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Texas,Female,Good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,No,...,1.68,113.40,40.35,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
1,Wisconsin,Female,Good,2.0,5.0,Within past year (anytime less than 12 months ...,Yes,8.0,1 to 5,No,...,1.73,104.33,34.97,Yes,Yes,No,Yes,"No, did not receive any tetanus shot in the pa...",No,No
2,Idaho,Male,Poor,7.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,1 to 5,No,...,1.73,104.33,34.97,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
3,Iowa,Female,Good,20.0,10.0,Within past year (anytime less than 12 months ...,No,8.0,All,Yes,...,1.68,127.01,45.19,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
4,South Dakota,Female,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.60,49.90,19.49,Yes,No,Yes,No,"Yes, received Tdap",No,Tested positive using home test without a heal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Utah,Male,Very good,3.0,0.0,5 or more years ago,Yes,7.0,None of them,No,...,1.70,77.11,26.63,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,No
99996,New Mexico,Male,Very good,30.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,All,Yes,...,1.88,90.72,25.68,No,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
99997,Michigan,Female,Very good,2.0,12.0,5 or more years ago,No,7.0,None of them,No,...,1.65,56.70,20.80,Yes,Yes,Yes,Yes,"Yes, received Tdap",Yes,No
99998,Arizona,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.91,106.59,29.37,No,No,Yes,No,"Yes, received tetanus shot, but not Tdap",No,No


In [ ]:
data_copy1 = data_copy.copy()
data_copy2 = data_copy.copy()

In [ ]:
numerical_cols = data_copy1.select_dtypes(include=['float64','number']).columns
numerical_data = data_copy1[numerical_cols]
numerical_data

,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI
0,0.0,0.0,8.0,1.68,113.40,40.35
1,2.0,5.0,8.0,1.73,104.33,34.97
2,7.0,10.0,6.0,1.73,104.33,34.97
3,20.0,10.0,8.0,1.68,127.01,45.19
4,0.0,0.0,7.0,1.60,49.90,19.49
...,...,...,...,...,...,...
99995,3.0,0.0,7.0,1.70,77.11,26.63
99996,30.0,10.0,6.0,1.88,90.72,25.68
99997,2.0,12.0,7.0,1.65,56.70,20.80
99998,0.0,0.0,7.0,1.91,106.59,29.37


In [ ]:
categorical_cols = data_copy1.select_dtypes(include=['object','category']).columns

In [ ]:
data_copy1['HeartConditionBinary'] = ((data_copy1['HadHeartAttack'] == 'Yes') | (data_copy1['HadAngina'] == 'Yes')).astype(int)
data_copy1

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,HeartConditionBinary
0,Texas,Female,Good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,No,...,113.40,40.35,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes,0
1,Wisconsin,Female,Good,2.0,5.0,Within past year (anytime less than 12 months ...,Yes,8.0,1 to 5,No,...,104.33,34.97,Yes,Yes,No,Yes,"No, did not receive any tetanus shot in the pa...",No,No,0
2,Idaho,Male,Poor,7.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,1 to 5,No,...,104.33,34.97,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No,1
3,Iowa,Female,Good,20.0,10.0,Within past year (anytime less than 12 months ...,No,8.0,All,Yes,...,127.01,45.19,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes,1
4,South Dakota,Female,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,49.90,19.49,Yes,No,Yes,No,"Yes, received Tdap",No,Tested positive using home test without a heal...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Utah,Male,Very good,3.0,0.0,5 or more years ago,Yes,7.0,None of them,No,...,77.11,26.63,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,No,0
99996,New Mexico,Male,Very good,30.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,All,Yes,...,90.72,25.68,No,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No,1
99997,Michigan,Female,Very good,2.0,12.0,5 or more years ago,No,7.0,None of them,No,...,56.70,20.80,Yes,Yes,Yes,Yes,"Yes, received Tdap",Yes,No,0
99998,Arizona,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,106.59,29.37,No,No,Yes,No,"Yes, received tetanus shot, but not Tdap",No,No,0


In [ ]:
data_copy1 = data_copy1.drop(['HadHeartAttack','HadAngina'],axis=1)

In [ ]:
data_copy1['HeartConditionBinary'] = data_copy1['HeartConditionBinary'].astype('int')
data_copy1['Sex'] = data_copy1['Sex'].astype('category')

In [ ]:
nominal_features = ['State', 'RaceEthnicityCategory', 'TetanusLast10Tdap', 'CovidPos','RemovedTeeth','SmokerStatus','ECigaretteUsage','HadDiabetes']
binary_features = [
     'HadStroke', 'HadAsthma', 'HadSkinCancer',
    'HadCOPD', 'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
    'DeafOrHardOfHearing', 'BlindOrVisionDifficulty', 'DifficultyConcentrating',
    'DifficultyWalking', 'DifficultyDressingBathing', 'DifficultyErrands',
    'PhysicalActivities', 'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12',
    'PneumoVaxEver', 'HighRiskLastYear', 'ChestScan'
]
ordinal_features = ['GeneralHealth', 'LastCheckupTime', 'AgeCategory']

In [ ]:
data_copy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 39 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   State                      100000 non-null  object  
 1   Sex                        100000 non-null  category
 2   GeneralHealth              100000 non-null  object  
 3   PhysicalHealthDays         100000 non-null  float64 
 4   MentalHealthDays           100000 non-null  float64 
 5   LastCheckupTime            100000 non-null  object  
 6   PhysicalActivities         100000 non-null  object  
 7   SleepHours                 100000 non-null  float64 
 8   RemovedTeeth               100000 non-null  object  
 9   HadStroke                  100000 non-null  object  
 10  HadAsthma                  100000 non-null  object  
 11  HadSkinCancer              100000 non-null  object  
 12  HadCOPD                    100000 non-null  object  
 13  HadDepressiveDi

In [ ]:
data_copy1['Sex'] = data_copy1['Sex'].replace({'Male':1,'Female':0})

one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
nominal_encoded = pd.DataFrame(
    one_hot_encoder.fit_transform(data_copy1[nominal_features]),
    columns=one_hot_encoder.get_feature_names_out(nominal_features),
    index=data_copy1.index  # Align indices with the original DataFrame
)

binary_encoded = data_copy1[binary_features].replace({'Yes': 1, 'No': 0}).astype(int)


ordinal_mappings = {
    'GeneralHealth': ['Poor', 'Fair', 'Good', 'Very good', 'Excellent'],
    'LastCheckupTime': [
        '5 or more years ago',
        'Within past 5 years (2 years but less than 5 years ago)',
        'Within past 2 years (1 year but less than 2 years ago)',
        'Within past year (anytime less than 12 months ago)'
    ],
    'AgeCategory':[
        'Age 18 to 24', 'Age 25 to 29', 'Age 30 to 34', 'Age 35 to 39', 'Age 40 to 44', 'Age 45 to 49', 'Age 50 to 54', 'Age 55 to 59','Age 60 to 64', 'Age 65 to 69'
         ,'Age 70 to 74','Age 75 to 79','Age 80 or older'
    ]
}

ordinal_encoded = data_copy1[ordinal_features].replace({
    col: {val: i for i, val in enumerate(vals)} for col, vals in ordinal_mappings.items()
})

<ipython-input-87-04b822447191>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_copy1['Sex'] = data_copy1['Sex'].replace({'Male':1,'Female':0})
<ipython-input-87-04b822447191>:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_copy1['Sex'] = data_copy1['Sex'].replace({'Male':1,'Female':0})
<ipython-input-87-04b822447191>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior,

In [ ]:
processed_data1 = pd.concat([nominal_encoded, binary_encoded, ordinal_encoded,numerical_data], axis=1)
processed_data1['HeartConditionBinary'] = data_copy1['HeartConditionBinary'].values

In [ ]:
X = processed_data1.drop(columns=['HeartConditionBinary'])
y = processed_data1['HeartConditionBinary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
X_train

,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_District of Columbia,State_Florida,...,ChestScan,GeneralHealth,LastCheckupTime,AgeCategory,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI
31152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,3,1,3.0,7.0,7.0,1.60,58.97,23.03
33499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,2,4,0.0,0.0,9.0,1.57,120.66,48.65
18594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,3,12,0.0,2.0,9.0,1.57,68.04,27.44
97132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,3,10,0.0,0.0,3.0,1.63,99.79,37.76
56015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,3,12,0.0,0.0,7.0,1.65,69.40,25.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,3,10,0.0,0.0,8.0,1.63,68.04,25.75
93553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,3,10,0.0,0.0,8.0,1.70,72.57,25.06
50496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,2,1,3.0,8.0,6.0,1.57,79.38,32.01
94735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,3,6,3.0,0.0,7.0,1.83,145.15,43.40


In [ ]:
X_train.columns = X_train.columns.str.replace(r'[^A-Za-z0-9_]', '_', regex=True)
X_test.columns = X_test.columns.str.replace(r'[^A-Za-z0-9_]', '_', regex=True)

In [ ]:
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (80000, 111) (80000,)
Test set shape: (20000, 111) (20000,)


In [ ]:
X_train

,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_District_of_Columbia,State_Florida,...,ChestScan,GeneralHealth,LastCheckupTime,AgeCategory,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI
31152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,3,1,3.0,7.0,7.0,1.60,58.97,23.03
33499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,2,4,0.0,0.0,9.0,1.57,120.66,48.65
18594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,3,12,0.0,2.0,9.0,1.57,68.04,27.44
97132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,3,10,0.0,0.0,3.0,1.63,99.79,37.76
56015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,3,12,0.0,0.0,7.0,1.65,69.40,25.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,3,10,0.0,0.0,8.0,1.63,68.04,25.75
93553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,3,10,0.0,0.0,8.0,1.70,72.57,25.06
50496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,2,1,3.0,8.0,6.0,1.57,79.38,32.01
94735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,3,3,6,3.0,0.0,7.0,1.83,145.15,43.40


#NO csl


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

hidden_size1 = 128
hidden_size2 = 64
dropout = 0.2

base_learners_nocsl = [
    ('rf', RandomForestClassifier(max_depth=25, min_samples_leaf=7, min_samples_split=18,
                       n_estimators=30, n_jobs=-1, random_state=10)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('lgbm', LGBMClassifier(feature_fraction=0.7101209541923168,
               learning_rate=0.026355528811002733, max_depth=8,
               min_child_samples=33, n_estimators=30, num_leaves=15,
               random_state=10)),
    ('mlp', MLPClassifier(
        hidden_layer_sizes=(hidden_size1, hidden_size2),  # Set the hidden layer sizes
        max_iter=75,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        random_state=10,
        alpha=dropout,
        solver='adam'))
    ]

meta_model_nocsl = LogisticRegression(C=0.00023921336045630725,max_iter=1000, random_state=10)

stacking_model_nocsl = StackingClassifier(
    estimators=base_learners_nocsl, final_estimator=meta_model_nocsl, cv=5)


In [ ]:
stacking_model_nocsl.fit(X_train, y_train)
y_pred_test_nocsl = stacking_model_nocsl.predict(X_test)

accuracy_test_nocsl = accuracy_score(y_test, y_pred_test_nocsl)
print(f'Accuracy of Stacked Generalization model: {accuracy_test_nocsl:.4f}')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:36:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Info] Number of positive: 6948, number of negative: 73052
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037770 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.086850 -> initscore=-2.352718
[LightGBM] [Info] Start training from score -2.352718


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:37:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:37:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:37:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:37:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:37:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Info] Number of positive: 5559, number of negative: 58441
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029363 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 879
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.086859 -> initscore=-2.352599
[LightGBM] [Info] Start training from score -2.352599
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=

In [ ]:
y_pred_train_nocsl = stacking_model_nocsl.predict(X_train)
accuracy_train_nocsl = accuracy_score(y_train, y_pred_train_nocsl)
print(f' Train accuracy of Stacked Generalization model: {accuracy_train_nocsl:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
 Train accuracy of Stacked Generalization model: 0.9100


In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test_nocsl))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_test_nocsl))


Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     18239
           1       0.00      0.00      0.00      1761

    accuracy                           0.91     20000
   macro avg       0.46      0.50      0.48     20000
weighted avg       0.83      0.91      0.87     20000


Confusion Matrix:
[[18239     0]
 [ 1761     0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import roc_auc_score

auc_roc_test_nocsl = roc_auc_score(y_test, y_pred_test_nocsl)
print(f"AUC-ROC: {auc_roc_test_nocsl}")

AUC-ROC: 0.5


#csl

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score

weight_list = [
    {0: 1, 1: 10.48},  # IR
    {0: (1/10.48), 1: 1},      # WCUSTOM WITH IR
    {0:0.00896487902392998, 1:0.9910351209760699},      # ICF
    {0:0.03584837416030507, 1:0.9641516258396949}       # LICF
]  #100k

'''weight_list = [
    {0: 1, 1: 10.58},
    {0: (1/10.58), 1: 1},
    {0:0.10308744939129459, 1:1.09421162},
    {0:0.12674908447458588, 1:1.0900341199999999}   #50k
]'''


'''weight_list = [
    {0: 1, 1: 10.40},
    {0: (1/10.40), 1: 1},
    {0:0.009153652845914623, 1:0.9908463471540853},
    {0:0.03633484727524011, 1:0.9636651527247599}
]'''   #20k

'''weight_list = [
    {0: 1, 1: 10.71},
    {0: (1/10.71), 1: 1},
    {0:0.008583156509528652, 1:0.9914168434904713},
    {0:0.034853779712300276, 1:0.9651462202876997}
] '''  #10k


base_learners = [
    ('rf', RandomForestClassifier(n_estimators=20, random_state=10)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('lgbm', LGBMClassifier(n_estimators=20, random_state=10)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(64,32), max_iter=50, early_stopping=True,
                          validation_fraction=0.1, n_iter_no_change=10, random_state=10))
]

# Initialize the stacking model's meta learner (Logistic Regression)
meta_model = LogisticRegression(C=0.00023921336045630725, max_iter=1000, random_state=10)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier

# Initialize the meta model (Logistic Regression)
meta_model = LogisticRegression(random_state=10)

# Initialize base learners (random forest, XGBoost, and LightGBM)
base_learners = [
    ('rf', RandomForestClassifier(n_estimators=20, random_state=10)),  # RandomForest
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),  # XGBoost
    ('lgbm', LGBMClassifier(n_estimators=20, random_state=10))  # LightGBM
]

# List to store results
results = []

# Loop through class weights
for cw in weight_list:
    # Update class weights for the base learners that support it
    base_learners[0] = ('rf', RandomForestClassifier(n_estimators=20, random_state=10, class_weight=cw))  # RandomForest
    base_learners[1] = ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=cw[1]/cw[0]))  # XGBoost
    base_learners[2] = ('lgbm', LGBMClassifier(n_estimators=20, random_state=10, class_weight=cw))  # LightGBM

    # Update class weight for the meta model (Logistic Regression)
    meta_model.set_params(class_weight=cw)

    # Initialize the stacking model with the current class weights for the base learners and meta model
    stacking_model = StackingClassifier(classifiers=[base_learners[0][1], base_learners[1][1], base_learners[2][1]],
                                       meta_classifier=meta_model)

    # Train the stacking model with the updated class weights for the base learners and meta model
    stacking_model.fit(X_train, y_train)

    # Predictions on the test set
    y_pred = stacking_model.predict(X_test)
    y_pred_prob = stacking_model.predict_proba(X_test)[:, 1]

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall_0 = recall_score(y_test, y_pred, pos_label=0)
    recall_1 = recall_score(y_test, y_pred, pos_label=1)
    f1_0 = f1_score(y_test, y_pred, pos_label=0)
    f1_1 = f1_score(y_test, y_pred, pos_label=1)
    auc_roc = roc_auc_score(y_test, y_pred_prob)

    # Store results for each class weight scenario with 4 decimal places
    results.append({
        'Class Weights': str(cw),
        'Accuracy': f"{accuracy:.4f}",
        'Recall Class 0': f"{recall_0:.4f}",
        'Recall Class 1': f"{recall_1:.4f}",
        'F1 Class 0': f"{f1_0:.4f}",
        'F1 Class 1': f"{f1_1:.4f}",
        'AUC-ROC': f"{auc_roc:.4f}"
    })

    # Print the results for each class weight scenario
    print(f"Class Weights: {cw}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall (Class 0): {recall_0:.4f}, Recall (Class 1): {recall_1:.4f}")
    print(f"F1 Score (Class 0): {f1_0:.4f}, F1 Score (Class 1): {f1_1:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print("-" * 50)

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display the results in the console
print("\nModel Evaluation Results:")
print(results_df)




/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:27:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 6948, number of negative: 73052
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038634 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499188 -> initscore=-0.003249
[LightGBM] [Info] Start training from score -0.003249
Class Weights: {0: 1, 1: 10.48}
Accuracy: 0.9128
Recall (Class 0): 0.9968, Recall (Class 1): 0.0432
F1 Score (Class 0): 0.9542, F1 Score (Class 1): 0.0802
AUC-ROC: 0.7000
--------------------------------------------------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:27:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 6948, number of negative: 73052
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038532 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499188 -> initscore=-0.003249
[LightGBM] [Info] Start training from score -0.003249
Class Weights: {0: 0.09541984732824427, 1: 1}
Accuracy: 0.9119
Recall (Class 0): 0.9967, Recall (Class 1): 0.0341
F1 Score (Class 0): 0.9538, F1 Score (Class 1): 0.0638
AUC-ROC: 0.6980
--------------------------------------------------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:28:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 6948, number of negative: 73052
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.913150 -> initscore=2.352718
[LightGBM] [Info] Start training from score 2.352718
Class Weights: {0: 0.00896487902392998, 1: 0.9910351209760699}
Accuracy: 0.5514
Recall (Class 0): 0.5181, Recall (Class 1): 0.8961
F1 Score (Class 0): 0.6781, F1 Score (Class 1): 0.2602
AUC-ROC: 0.7260
--------------------------------------------------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:28:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 6948, number of negative: 73052
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055831 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.718945 -> initscore=0.939233
[LightGBM] [Info] Start training from score 0.939233
Class Weights: {0: 0.03584837416030507, 1: 0.9641516258396949}
Accuracy: 0.9070
Recall (Class 0): 0.9906, Recall (Class 1): 0.0403
F1 Score (Class 0): 0.9510, F1 Score (Class 1): 0.0709
AUC-ROC: 0.7129
--------------------------------------------------

Model Evaluation Results:
                                     Class Weights Accuracy Recall Class 0  \
0                                 {0: 1, 1: 10.48}   0.9128         0.9968   
1        

In [ ]:
for cw in weight_list:
    # Update class weights for the base learners that support it
    base_learners[0] = ('rf', RandomForestClassifier(n_estimators=20, random_state=10, class_weight=cw))  # RandomForest
    base_learners[1] = ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=cw[1]/cw[0]))  # XGBoost
    base_learners[2] = ('lgbm', LGBMClassifier(n_estimators=20, random_state=10, class_weight=cw))  # LightGBM

    # Update class weight for the meta model (Logistic Regression)
    meta_model.set_params(class_weight=cw)

    # Initialize the stacking model with the current class weights for the base learners and meta model
    stacking_model = StackingClassifier(estimators=base_learners, final_estimator=meta_model, cv=5)

    # Train the stacking model with the updated class weights for the base learners and meta model
    stacking_model.fit(X_train, y_train)

    # Predictions on the test set
    y_pred = stacking_model.predict(X_test)
    y_pred_prob = stacking_model.predict_proba(X_test)[:, 1]

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall_0 = recall_score(y_test, y_pred, pos_label=0)
    recall_1 = recall_score(y_test, y_pred, pos_label=1)
    f1_0 = f1_score(y_test, y_pred, pos_label=0)
    f1_1 = f1_score(y_test, y_pred, pos_label=1)
    auc_roc = roc_auc_score(y_test, y_pred_prob)

    # Print the results for each class weight scenario
    print(f"Class Weights: {cw}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall (Class 0): {recall_0:.4f}, Recall (Class 1): {recall_1:.4f}")
    print(f"F1 Score (Class 0): {f1_0:.4f}, F1 Score (Class 1): {f1_1:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print("-" * 50

In [ ]:
'''from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

class_weights_lgm = {0:0.009086077881960733, 1:0.9909139221180393}
class_weights_xg = {0:0.009086077881960733, 1:0.9909139221180393}

hidden_size1 = 128
hidden_size2 = 64
dropout = 0.2

base_learners = [
    ('rf', RandomForestClassifier(class_weight={0: 0.09633911368015413, 1: 1},
                       max_depth=25, min_samples_leaf=7, min_samples_split=18,
                       n_estimators=160, n_jobs=-1, random_state=10)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss',scale_pos_weight=class_weights_xg[1])),
    ('lgbm', LGBMClassifier(class_weight=class_weights_lgm,feature_fraction=0.7101209541923168,
               learning_rate=0.026355528811002733, max_depth=8,
               min_child_samples=33, n_estimators=137, num_leaves=38,
               random_state=10)),
    ('mlp', MLPClassifier(
        hidden_layer_sizes=(hidden_size1, hidden_size2),  # Set the hidden layer sizes
        max_iter=75,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        random_state=10,
        alpha=dropout,
        solver='adam'))
]

meta_model = LogisticRegression(C=0.00023921336045630725, class_weight={0: 1, 1: 10.38},
                   max_iter=1000, random_state=10)

stacking_model = StackingClassifier(
    estimators=base_learners, final_estimator=meta_model, cv=5)'''


In [ ]:
'''stacking_model.fit(X_train, y_train)
y_pred = stacking_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of Stacked Generalization model: {accuracy:.4f}')'''

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:54:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Info] Number of positive: 17199, number of negative: 179611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.092611 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 905
[LightGBM] [Info] Number of data points in the train set: 196810, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.912611 -> initscore=2.345942
[LightGBM] [Info] Start training from score 2.345942


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:59:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:59:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:59:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:59:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:00:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Info] Number of positive: 13760, number of negative: 143688
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073595 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 898
[LightGBM] [Info] Number of data points in the train set: 157448, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.912616 -> initscore=2.346006
[LightGBM] [Info] Start training from score 2.346006
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction

In [ ]:
'''y_pred_train = stacking_model.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f' Train accuracy of Stacked Generalization model: {accuracy_train:.4f}')''''

[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
 Train accuracy of Stacked Generalization model: 0.7565


In [ ]:
'''print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))'''


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.73      0.84     44786
           1       0.23      0.81      0.36      4417

    accuracy                           0.74     49203
   macro avg       0.60      0.77      0.60     49203
weighted avg       0.91      0.74      0.79     49203


Confusion Matrix:
[[32834 11952]
 [  853  3564]]


#SMOTE

In [ ]:
from sklearn.datasets import make_classification
from collections import Counter

class_counts = Counter(y)

y_0 = class_counts[0]
y_1 = class_counts[1]

print(f"Class 0 count: {y_0}")
print(f"Class 1 count: {y_1}")

Class 0 count: 4555
Class 1 count: 445


In [ ]:
X1_train = X_train.copy()
y1_train = y_train.copy()
X1_test = X_test.copy()
y1_test = y_test.copy()

In [ ]:
class_counts_train = Counter(y1_train)
y1_train_0 = class_counts_train[0]
y1_train_1 = class_counts_train[1]

print(f"Class 0 count train: {y1_train_0}")
print(f"Class 1 count train: {y1_train_1}")


class_counts_test = Counter(y1_test)
y1_test_0 = class_counts_test[0]
y1_test_1 = class_counts_test[1]

print(f"Class 0 count test: {y1_test_0}")
print(f"Class 1 count test: {y1_test_1}")

Class 0 count train: 3640
Class 1 count train: 360
Class 0 count test: 915
Class 1 count test: 85


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=10)
X1s_train, y1s_train = smote.fit_resample(X1_train, y1_train)

print("After SMOTE with Tomek Links:")
print("Original class distribution:", Counter(y1_train))
print("Resampled class distribution:", Counter(y1s_train))

After SMOTE with Tomek Links:
Original class distribution: Counter({0: 3640, 1: 360})
Resampled class distribution: Counter({0: 3640, 1: 3640})


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

hidden_size1 = 128
hidden_size2 = 64
dropout = 0.2

base_learners_smote = [
    ('rf', RandomForestClassifier(max_depth=25, min_samples_leaf=7, min_samples_split=18,
                       n_estimators=160, n_jobs=-1, random_state=10)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('lgbm', LGBMClassifier(feature_fraction=0.7101209541923168,
               learning_rate=0.026355528811002733, max_depth=8,
               min_child_samples=33, n_estimators=137, num_leaves=38,
               random_state=10)),
    ('mlp', MLPClassifier(
        hidden_layer_sizes=(hidden_size1, hidden_size2),  # Set the hidden layer sizes
        max_iter=75,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        random_state=10,
        alpha=dropout,
        solver='adam'))
    ]

meta_model_smote = LogisticRegression(C=0.00023921336045630725,max_iter=1000, random_state=10)

stacking_model_smote = StackingClassifier(
    estimators=base_learners_smote, final_estimator=meta_model_smote, cv=5)


In [ ]:
stacking_model_smote.fit(X1s_train, y1s_train)
y_pred_test_smote = stacking_model_smote.predict(X_test)

accuracy_test_smote = accuracy_score(y1_test, y_pred_test_smote)
print(f'Accuracy of Stacked Generalization model: {accuracy_test_smote:.4f}')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:38:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Info] Number of positive: 3640, number of negative: 3640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006795 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9458
[LightGBM] [Info] Number of data points in the train set: 7280, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:38:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:38:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:38:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:38:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:38:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Info] Number of positive: 2912, number of negative: 2912
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005693 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8937
[LightGBM] [Info] Number of data points in the train set: 5824, number of used features: 110
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
[LightGBM] [Warning] feature_fraction

In [ ]:
y_pred_train_smote = stacking_model_smote.predict(X1s_train)
accuracy_train_smote = accuracy_score(y1s_train, y_pred_train_smote)
print(f' Train accuracy of Stacked Generalization model: {accuracy_train_smote:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.7101209541923168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7101209541923168
 Train accuracy of Stacked Generalization model: 0.9848


In [ ]:
print("\nClassification Report:")
print(classification_report(y1_test, y_pred_test_smote))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y1_test, y_pred_test_smote))


Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       915
           1       0.34      0.12      0.18        85

    accuracy                           0.91      1000
   macro avg       0.63      0.55      0.56      1000
weighted avg       0.87      0.91      0.88      1000


Confusion Matrix:
[[896  19]
 [ 75  10]]


In [ ]:
from sklearn.metrics import roc_auc_score

auc_roc_test_smote = roc_auc_score(y1_test, y_pred_test_smote)
print(f"AUC-ROC: {auc_roc_test_smote}")

AUC-ROC: 0.5484410157505626
